In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier

# Charger les données
train_data = pd.read_csv('/home/charlemagne/workspace/kaggle_challenge_titanic/data/train.csv')
test_data = pd.read_csv('/home/charlemagne/workspace/kaggle_challenge_titanic/data/test.csv')
submission_file_path = '/home/charlemagne/workspace/kaggle_challenge_titanic/submission/dumb_test_complet.csv'

# Fonction de prétraitement
def preprocess_data(data, columns=None):
    data['Age'].fillna(data['Age'].median(), inplace=True)
    data['Embarked'].fillna(data['Embarked'].mode()[0], inplace=True)
    data['Fare'].fillna(data['Fare'].median(), inplace=True)
    data = pd.get_dummies(data, columns=['Sex', 'Embarked', 'Pclass'])
    data['FamilySize'] = data['SibSp'] + data['Parch'] + 1
    data.drop(['Ticket', 'Cabin', 'Name', 'SibSp', 'Parch'], axis=1, inplace=True)

    # Aligner les colonnes avec l'ensemble de référence
    if columns is not None:
        data = data.reindex(columns=columns, fill_value=0)
    return data

# Conserver PassengerId pour le fichier de soumission
passenger_id = test_data['PassengerId']

# Prétraitement des données d'entraînement
train_data = preprocess_data(train_data)
X = train_data.drop(["Survived", "PassengerId"], axis=1)
y = train_data["Survived"]
X_train_columns = X.columns

# Prétraitement des données de test
test_data = preprocess_data(test_data, columns=X_train_columns)

# Séparer les données d'entraînement
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Définir les modèles
rf = RandomForestClassifier(random_state=42)
xgb = XGBClassifier(random_state=42)

# Grilles de paramètres pour le réglage
param_grid_rf = {'n_estimators': [100, 200], 'max_depth': [5, 10]}
param_grid_xgb = {'n_estimators': [100, 200], 'learning_rate': [0.05, 0.1]}

# GridSearchCV pour le réglage des hyperparamètres
grid_rf = GridSearchCV(estimator=rf, param_grid=param_grid_rf, cv=5, scoring='accuracy')
grid_xgb = GridSearchCV(estimator=xgb, param_grid=param_grid_xgb, cv=5, scoring='accuracy')

# Entraînement des modèles
grid_rf.fit(X_train, y_train)
grid_xgb.fit(X_train, y_train)

# Classifier de vote
voting_clf = VotingClassifier(estimators=[('rf', grid_rf.best_estimator_), ('xgb', grid_xgb.best_estimator_)], voting='soft')
voting_clf.fit(X_train, y_train)

# Évaluation sur l'ensemble de validation
y_pred = voting_clf.predict(X_val)
print(f"Validation Accuracy: {accuracy_score(y_val, y_pred)}")

# Prédiction sur l'ensemble de test
test_predictions = voting_clf.predict(test_data)

# Créer le fichier de soumission
submission = pd.DataFrame({'PassengerId': passenger_id, 'Survived': test_predictions})
submission.to_csv(submission_file_path, index=False)


Validation Accuracy: 0.8156424581005587
